Assignment 4 
<br>Using provided csv data and wikipedia page (that you must scrape), determine if there is a correlation between the size of a city and the success of NHL teams.  

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    #data cleaning for NHL
    nhl_df=pd.read_csv("nhl.csv")
    nhl2018 = nhl_df.where(nhl_df['year'] == int('2018')).dropna()
    nhl2018 = nhl2018.drop(index=[0,9,18,26])
    nhl2018['W'] = nhl2018['W'].astype('float64')
    nhl2018['L'] = nhl2018['L'].astype('float64')
    nhl2018['W-L%'] = nhl2018.apply(lambda x: x['W']/(x['W']+x['L']), axis=1)
    nhl2018 = nhl2018.reset_index(drop=True)
    nhl2018 = nhl2018.iloc[ : , [0,-1]]
    nhl2018 = nhl2018.replace(to_replace='[\*\+]', value='', regex=True)
    
    #data cleaning for cities df
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.replace(to_replace='\[.*\]', value='', regex=True)
    
    #function for converting teams string values in cities df into lists
    def team_listtest(x):
        teamlstsp = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+', x)
        teamlstwh = re.findall('[A-Z0-9][0-9a-z]+', x)
        if len(teamlstsp) > 0 and len(teamlstwh) > 2:
            teamlst = [teamlstsp[0],teamlstwh[0]]
        elif len(teamlstsp) < 1:
            teamlst = teamlstwh
        elif len(teamlstsp) > 0 and len(teamlstwh) <= 2:
            teamlst = teamlstsp
        return teamlst
    
    #run that function in cities
    cities['NHL'] = cities['NHL'].apply(team_listtest)
    
    #function for getting team name out of nhl2018 df, then comparing it to the team list in cities, and building a new series of W-L%
    def team_get(x,y,z):
        allwllist = list()
        for teamlist in y:
            wllist = list()
            value = 0
            for team in x:
                teamname = re.findall('\s([A-Z0-9][a-z0-9]+)$', team)
                teamname = teamname[0]
                if teamname in ['Sox', 'Leafs', 'Knights', 'Devils', 'Blazers', 'Wings', 'Jackets', 'Jays']:
                    teamname = re.findall('([A-Z][a-z]+\s[A-Z0-9][a-z0-9]+)$', team)
                    teamname = teamname[0]
                if teamname in teamlist:
                    #print(teamname, teamlist)
                    i = x.loc[x == team].index[0]
                    value = float(z[i])
                    wllist.append(value)
                else: continue
            try: avgvalue = sum(wllist)/len(wllist)
            except: avgvalue = np.nan
            allwllist.append(avgvalue)
        return allwllist 
    
    #run that function to produce a new Series
    cities['NHLwinloss'] = pd.Series(team_get(nhl2018['team'], cities['NHL'], nhl2018['W-L%']))
    
    #make a new df of only populations and NHLwinloss to dropna
    popwlNHL = cities.loc[ : ,['Population (2016 est.)[8]','NHLwinloss']]
    popwlNHL = popwlNHL.dropna()
    
    #raise NotImplementedError()
    
    population_by_region = popwlNHL['Population (2016 est.)[8]'].astype('float64') 
    # pass in metropolitan area population from cities
    win_loss_by_region = popwlNHL['NHLwinloss'] 
    # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
#     assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
nhl_correlation()

PearsonRResult(statistic=-0.04027318804589654, pvalue=0.8387661496942489)

In [ ]:
### THIS CODE WILL NOT WORK IN THIS JUPYTER ENVIRONMENT, just copying for safety sake
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    #data cleaning for NHL
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl2018 = nhl_df.where(nhl_df['year'] == int('2018')).dropna()
    nhl2018 = nhl2018.drop(index=[0,9,18,26])
    nhl2018['W'] = nhl2018['W'].astype('float64')
    nhl2018['L'] = nhl2018['L'].astype('float64')
    nhl2018['W-L%'] = nhl2018.apply(lambda x: x['W']/(x['W']+x['L']), axis=1)
    nhl2018 = nhl2018.reset_index(drop=True)
    nhl2018 = nhl2018.iloc[ : , [0,-1]]
    nhl2018 = nhl2018.replace(to_replace='[\*\+]', value='', regex=True)
    
    #data cleaning for cities df
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.replace(to_replace='\[.*\]', value='', regex=True)
    
    #special regex to compensate for the difference in parsing html between this juptyer environemnt and my local jupyter environement
    #the difference in parsing makes my strategy for completing the assignment useless
    #specifically because in this Jupyter environemnt, spaces are inserted between the team names in the cities df, whereas in my local environment,
    #there are no spaces, except when the team name is two words.
    #I am compensating manually below, thinking its easier just to get the cities df to look like it did when I wrote my whole program
    #this is very frustrating!
    
    cities['NHL'] = cities['NHL'].replace(to_replace='\s', value='', regex=True)

    #function for converting teams string values in cities df into lists
    #I had to modify this in a really awkward way again to compensate for the above discrepancy in how the different Juptyer environemnts parse html
    def team_listtest(x):
        teamlstsp = re.findall('[A-Z][a-z]+\s[A-Z][a-z]+', x)
        teamlstwh = re.findall('[A-Z0-9][0-9a-z]+', x)
        if len(teamlstsp) > 0 and len(teamlstwh) > 2:
            teamlst = [teamlstsp[0],teamlstwh[0]]
        elif len(teamlstsp) < 1:
            teamlst = teamlstwh
        elif len(teamlstsp) > 0 and len(teamlstwh) <= 2:
            teamlst = teamlstsp
        if len(teamlst)>1 and teamlst[1] in ['Sox', 'Leafs', 'Knights', 'Devils', 'Blazers', 'Wings', 'Jackets', 'Jays']:
            teamlst = [teamlst[0]+' '+teamlst[1]]
        if len(teamlst)>1 and teamlst[0] == 'Cubs':
            teamlst = ['Cubs', 'White Sox']
        return teamlst
    
    #run that function in cities
    cities['NHL'] = cities['NHL'].apply(team_listtest)
    
    #function for getting team name out of nhl2018 df, then comparing it to the team list in cities, and building a new series of W-L%
    def team_get(x,y,z):
        allwllist = list()
        for teamlist in y:
            wllist = list()
            value = 0
            for team in x:
                teamname = re.findall('\s([A-Z0-9][a-z0-9]+)$', team)
                teamname = teamname[0]
                if teamname in ['Sox', 'Leafs', 'Knights', 'Devils', 'Blazers', 'Wings', 'Jackets', 'Jays']:
                    teamname = re.findall('([A-Z][a-z]+\s[A-Z0-9][a-z0-9]+)$', team)
                    teamname = teamname[0]
                if teamname in teamlist:
                    #print(teamname, teamlist)
                    i = x.loc[x == team].index[0]
                    value = float(z[i])
                    wllist.append(value)
                else: continue
            try: avgvalue = sum(wllist)/len(wllist)
            except: avgvalue = np.nan
            allwllist.append(avgvalue)
        return allwllist 
    
    #run that function to produce a new Series
    cities['NHLwinloss'] = pd.Series(team_get(nhl2018['team'], cities['NHL'], nhl2018['W-L%']))
    
    #make a new df of only populations and NHLwinloss to dropna
    popwlNHL = cities.loc[ : ,['Population (2016 est.)[8]','NHLwinloss']]
    popwlNHL = popwlNHL.dropna()
    
    #raise NotImplementedError()
    
    population_by_region = popwlNHL['Population (2016 est.)[8]'].astype('float64') 
    # pass in metropolitan area population from cities
    win_loss_by_region = popwlNHL['NHLwinloss'] 
    # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
nhl_correlation()